In [1]:
##############      Configuración      ##############
import gc, sys, os

from dotenv import dotenv_values
from plantilla import plantilla
import numpy as np
import pandas as pd
import geopandas as gpd

import cartopy.crs as crs
import cartopy.io.img_tiles as cimgt
from matplotlib.figure import Figure
from shapely.geometry import box
from PIL import Image


# assert (
#     len(sys.argv) == 4
# ), "Se esperan 3 argumentos: 1) Parte, 2) Total de partes, 3) Map Provider: ESRI o GMaps"
parte = 1
total_partes = 8
map_name = "ESRI"

env = dotenv_values(
    r"R:\Tesis Nico\Códigos\scripts\Testing - Deep Learning con MapTilesDownloader\globals.env"
)

proyecto = "Códigos"
subproyecto = "Testing - Deep Learning con MapTilesDownloader"

globales = plantilla(
    proyecto=proyecto, subproyecto=subproyecto, path_proyectos=env["PATH_PROYECTOS"]
)

path_proyecto = globales[0]  # Ubicación de la carpeta del Proyecto
path_datain = globales[1]
path_dataout = globales[2]  # Bases procesadas por tus scripts
path_scripts = globales[3]
path_figures = globales[4]  # Output para las figuras/gráficos
path_maps = globales[5]  # Output para los mapas (html o imagen)
path_tables = globales[6]  # Output para las tablas (imagen o excel)
path_programas = globales[7]


icpag = gpd.read_file(
    r"R:\Shapefiles\ICPAG\Sin barrios pop y cerr\Aglomerados de mas de 500k habitantes\base_icpag_500k.shp"
)
icpag = icpag.to_crs(epsg=3857)
# icpag = icpag[icpag.area <= 1_000_000]  # Aprox p90
# icpag_amba = icpag[icpag.AMBA_legal == 1]
icpag = icpag.reset_index(drop=True)



R:/Tesis Nico\Códigos\scripts\Testing - Deep Learning con MapTilesDownloader
El directorio en scripts existe. Creando carpetas.
R:/Tesis Nico\Códigos\data ya existe
R:/Tesis Nico\Códigos\data\data_in ya existe
R:/Tesis Nico\Códigos\data\data_out ya existe
R:/Tesis Nico\Códigos\data\data_out\Testing - Deep Learning con MapTilesDownloader ya existe
R:/Tesis Nico\Códigos\docs ya existe
R:/Tesis Nico\Códigos\scripts ya existe
R:/Tesis Nico\Códigos\scripts\Testing - Deep Learning con MapTilesDownloader ya existe
R:/Tesis Nico\Códigos\outputs ya existe
R:/Tesis Nico\Códigos\outputs\figures ya existe
R:/Tesis Nico\Códigos\outputs\figures\Testing - Deep Learning con MapTilesDownloader ya existe
R:/Tesis Nico\Códigos\outputs\maps ya existe
R:/Tesis Nico\Códigos\outputs\maps\Testing - Deep Learning con MapTilesDownloader ya existe
R:/Tesis Nico\Códigos\outputs\tables ya existe
R:/Tesis Nico\Códigos\outputs\tables\Testing - Deep Learning con MapTilesDownloader ya existe


In [5]:
type(icpag.geometry.unary_union)

shapely.geometry.multipolygon.MultiPolygon

In [7]:
from rasterio import features
asd = features.rasterize(icpag.geometry.unary_union, out_shape=(100, 100))

C:\Users\Administrador\AppData\Roaming\Python\Python39\site-packages\rasterio\features.py:288: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for index, item in enumerate(shapes):


In [97]:
def random_point_from_geometry(gdf_slice):
    """Generates a random point within the bounds of a GeoDataFrame."""

    gdf_slice = gdf_slice.copy()

    # Get bounds of the shapefile's polygon
    bbox = gdf_slice.bounds

    while 0 == 0:
        # generate random data within the bounds
        x = np.random.uniform(bbox["minx"], bbox["maxx"], 1)
        y = np.random.uniform(bbox["miny"], bbox["maxy"], 1)

        # convert them to a points GeoSeries
        gdf_points = gpd.GeoSeries(gpd.points_from_xy(x, y))
        # only keep those points within polygons
        gdf_points = gdf_points[gdf_points.within(gdf_slice.unary_union)]
        if len(gdf_points) > 0:
            break
    
    print(x, y)
    gdf_slice = gdf_slice.drop(columns='geometry')
    gdf_slice = gpd.GeoDataFrame(gdf_slice,geometry=gpd.points_from_xy(x, y))
    gdf_slice = gdf_slice.set_crs(epsg=3857)
    gdf_slice.geometry = gdf_slice.centroid.buffer(100)
    
    return gdf_slice


def create_map_from_geometry(
    icpag,
    index,
    map_provider=map_provider,
    zoom=18,
    map_name="ESRI_WI",
    path_output=path_datain,
    sample_size=1,
    my_dpi=96,
):

    """Exporto imgs de 512x512.

    Matplotlib doesn't work with pixels directly, but rather physical sizes and DPI.
    If you want to display a figure with a certain pixel size, you need to know the DPI of your monitor.
    For example this link (https://www.infobyip.com/detectmonitordpi.php) will detect that for you.
    """

    for i in range(0, sample_size):
        try:
            # Reduzco el polygono para que sea aprox una manzana
            polygon = random_point_from_geometry(icpag.iloc[index:index+1,:])
            # polygon = icpag.iloc[index : index + 1, :]
            print(polygon.geometry)
            # Genero la máscara para el gráfico y obtengo el extent
            link = polygon.at[index, "link"]
            bbox = polygon.bounds
            geom = box(*bbox.values[0])
            mask = polygon.copy()
            mask["geometry"] = geom

            # Gráfico
            # The pylab figure manager will be bypassed in this instance.
            # This means that `fig` will be garbage collected as you'd expect.
            fig = Figure(dpi=my_dpi, figsize=(512 / my_dpi, 512 / my_dpi), linewidth=0)
            ax = fig.add_axes(
                [0, 0, 1, 1], projection=crs.epsg(3857), facecolor="black"
            )

            # Limita la visualización a un área determinada
            ax.set_extent(
                [bbox["minx"], bbox["maxx"], bbox["miny"], bbox["maxy"]],
                crs=crs.epsg(3857),
            )

            # Agrego mapa de fondo
            ax.add_image(map_provider, zoom)

            # Quito bordes y grilla
            ax.set(frame_on=False)

            # # Añade la máscara
            # mask.difference(polygon).plot(ax=ax, facecolor='black', edgecolor='black', linewidth=0.0)
            # fig.add_axes(ax)

            fig.savefig(
                rf"{path_output}\Imagenes descargadas\{map_name}_{link}_{index}_{i}.tiff",
                dpi=my_dpi,
            )
            del fig
            gc.collect()

        except Exception as e:
            print(e)
            continue

In [98]:
gdf_slice = icpag.iloc[index:index+1,:]

gdf_slice = gdf_slice.copy()

# Get bounds of the shapefile's polygon
bbox = gdf_slice.bounds

while 0 == 0:
    # generate random data within the bounds
    x = np.random.uniform(bbox["minx"], bbox["maxx"], 1)
    y = np.random.uniform(bbox["miny"], bbox["maxy"], 1)

    # convert them to a points GeoSeries
    gdf_points = gpd.GeoSeries(gpd.points_from_xy(x, y))
    # only keep those points within polygons
    gdf_points = gdf_points[gdf_points.within(gdf_slice.unary_union)]

    if len(gdf_points) > 0:
        break
print(gdf_points)

# polygon = polygon.set_crs(epsg=3857)
# gdf_slice['geometry'] = gpd.GeoSeries([gdf_points])
# gdf_slice.loc[:, "geometry"]
gdf_slice = gdf_slice.drop(columns='geometry')
gdf_slice = gpd.GeoDataFrame(gdf_slice,geometry=gpd.points_from_xy(x, y))
gdf_slice = gdf_slice.set_crs(epsg=3857)
gdf_slice

0    POINT (-6499577.552 -4106806.400)
dtype: geometry


,link,AREA,PERIMETER,PAISXRAD10,PAISXRAD_1,PROV,DEPTO,FRAC,RADIO,TIPO,...,total,total_val,con_nbi,sin_nbi,nbi_rc,nbi_rc_val,icpagNabs,CicpagNv,icpag,geometry
2,020010202,17843.625,770.726,31544,31544,02,001,02,02,U,...,132,128.0,26,102,0.19697,0.203125,-1.224758,7.0,0.07,POINT (-6499577.552 -4106806.400)


In [99]:
index = 2

polygon = random_point_from_geometry(icpag.iloc[index:index+1,:])
# polygon = icpag.iloc[index : index + 1, :]
print(polygon.geometry)
# Genero la máscara para el gráfico y obtengo el extent
link = polygon.at[index, "link"]
bbox = polygon.bounds
geom = box(*bbox.values[0])
mask = polygon.copy()
mask["geometry"] = geom

# Gráfico
# The pylab figure manager will be bypassed in this instance.
# This means that `fig` will be garbage collected as you'd expect.
fig = Figure(dpi=my_dpi, figsize=(512 / my_dpi, 512 / my_dpi), linewidth=0)
ax = fig.add_axes(
    [0, 0, 1, 1], projection=crs.epsg(3857), facecolor="black"
)

# Limita la visualización a un área determinada
ax.set_extent(
    [bbox["minx"], bbox["maxx"], bbox["miny"], bbox["maxy"]],
    crs=crs.epsg(3857),
)

# Agrego mapa de fondo
ax.add_image(map_provider, zoom)

# Quito bordes y grilla
ax.set(frame_on=False)

# # Añade la máscara
# mask.difference(polygon).plot(ax=ax, facecolor='black', edgecolor='black', linewidth=0.0)
# fig.add_axes(ax)

fig.savefig(
    rf"{path_output}\Imagenes descargadas\{map_name}_{link}_{index}_{i}.tiff",
    dpi=my_dpi,
)
del fig
gc.collect()


[-6499656.15645086] [-4106897.09299506]
2    POLYGON ((-6499556.156 -4106897.093, -6499556....
Name: geometry, dtype: geometry


NameError: name 'my_dpi' is not defined

In [94]:
gdf_slice

,link,AREA,PERIMETER,PAISXRAD10,PAISXRAD_1,PROV,DEPTO,FRAC,RADIO,TIPO,...,total,total_val,con_nbi,sin_nbi,nbi_rc,nbi_rc_val,icpagNabs,CicpagNv,icpag,geometry
1,020010201,27667.875,921.09,31535,31535,02,001,02,01,U,...,162,161.0,43,118,0.265432,0.267081,-1.19917,7.0,0.07,POINT (-6499576.137 -4106852.577)


In [101]:
create_map_from_geometry(icpag, 5, map_name=map_name, sample_size=5)

[-6499396.28766887] [-4107048.39840684]
5    POLYGON ((-6499296.288 -4107048.398, -6499296....
Name: geometry, dtype: geometry
[-6499241.65725692] [-4107038.89340867]
5    POLYGON ((-6499141.657 -4107038.893, -6499142....
Name: geometry, dtype: geometry
[-6499330.3034522] [-4107085.02639379]
5    POLYGON ((-6499230.303 -4107085.026, -6499230....
Name: geometry, dtype: geometry
[-6499290.94364775] [-4106998.72437674]
5    POLYGON ((-6499190.944 -4106998.724, -6499191....
Name: geometry, dtype: geometry
[-6499208.85525054] [-4107080.94159519]
5    POLYGON ((-6499108.855 -4107080.942, -6499109....
Name: geometry, dtype: geometry
